In [ ]:
本程序提供标准的针对本次数据集的数据eda，请在本程序提供的接口下进行data analysis，不要重新创建新的ipdb程序(在本程序中完成分析工作)
训练集和测试集数据均以dataframe格式给出，pytorch自定义datasets类接受dataframe的索引以npy格式保存，分析后的分类型训练的索引需要在filter_dbz.py
中修改，这里只给出demo

In [20]:
#以下程序提供一个简单的数据 demo
import pandas as pd
train_pd = '/media/workdir/hujh/hujh-new/huaweirader_baseline/data_eda/train_all_pkl.pkl'
test_pd = '/media/workdir/hujh/hujh-new/huaweirader_baseline/data_eda/test_all_pkl.pkl'

train_df = pd.read_pickle(train_pd)
test_df = pd.read_pickle(test_pd)

In [21]:
#sample imformation excavate
print(train_df.shape)
print(train_df.head(10))
train_df.describe()
print("------------------------")
print(test_df.shape)
print(test_df.head(10))
test_df.describe()

(820000, 1)
                                               fname
0  /media/data/huaweiraderdata/train/train_all/RA...
1  /media/data/huaweiraderdata/train/train_all/RA...
2  /media/data/huaweiraderdata/train/train_all/RA...
3  /media/data/huaweiraderdata/train/train_all/RA...
4  /media/data/huaweiraderdata/train/train_all/RA...
5  /media/data/huaweiraderdata/train/train_all/RA...
6  /media/data/huaweiraderdata/train/train_all/RA...
7  /media/data/huaweiraderdata/train/train_all/RA...
8  /media/data/huaweiraderdata/train/train_all/RA...
9  /media/data/huaweiraderdata/train/train_all/RA...
------------------------
(63000, 1)
                                               fname
0  /media/data/huaweiraderdata/test/test_all/RAD_...
1  /media/data/huaweiraderdata/test/test_all/RAD_...
2  /media/data/huaweiraderdata/test/test_all/RAD_...
3  /media/data/huaweiraderdata/test/test_all/RAD_...
4  /media/data/huaweiraderdata/test/test_all/RAD_...
5  /media/data/huaweiraderdata/test/test_all/RAD_..

,fname
count,63000
unique,63000
top,/media/data/huaweiraderdata/test/test_all/RAD_...
freq,1


In [22]:
train_df.values[0]

array(['/media/data/huaweiraderdata/train/train_all/RAD_184392561770451/RAD_184392561770451_000.png'],
      dtype=object)

In [48]:
train_df['fname'].keys()[0][0]

'RAD_184392561770451'

In [23]:
#二次检查排序方式
checkorigan_df = train_df
checksort_df = train_df.sort_values(by=['fname'])
assert (checkorigan_df.values == checksort_df .values).all()

In [24]:
#重新制索引用来聚类分析
train_index = list(map(lambda x:x[0].split('/')[-2],train_df.values))
train_df.index = [train_index,train_index]
train_df.index.names=['index0','index1']

test_index = list(map(lambda x:x[0].split('/')[-2],test_df.values))
test_df.index = [test_index,test_index]
test_df.index.names=['index0','index1']

In [37]:
train_df.values[0]

array(['/media/data/huaweiraderdata/train/train_all/RAD_184392561770451/RAD_184392561770451_000.png'],
      dtype=object)

In [26]:
#三次检查每一组样本是否都是41个文件，不是则drop
df = train_df.groupby(['index1']).describe().reset_index()
df[df['fname']['count'].values != 41].index
df.loc[lambda df: df['fname']['count'].values != 41, :]
#说明本次数据集无缺失文件

Empty DataFrame
Columns: [(index1, ), (fname, count), (fname, unique), (fname, top), (fname, freq)]
Index: []

In [34]:
#下面给出最简单的返回训练索引的demo，这个函数封装在filter_dbz中，数据eda需要分析后对返回的训练数据索引进行分类或者其他操作，分析后对
#对filter_dbz类的方法进行扩充，这里只给出最简单的demo---即不进行数据分型所有样本均参与训练
#即用前两个小时的数据预测后两个小时，input的长度为10间隔为12分钟，所以每个样本只返回初始时刻的下标索引t0
#整个样本seq input(t0，t0+12,t0+24,...t0+120),target(t0+150min,t0+180min,t0+220min,t0+240min)的加载在datasets.py中进行
from pathos.multiprocessing import ProcessingPool as Pool
#字符串--》41个样本对应的序号范围0-40
def splitstr_int(var):
    if var[0].split('/')[-1].split('.')[0][-2] == 0:
        var = int(var[0].split('/')[-1].split('.')[0][-1:])
    else:
        var = int(var[0].split('/')[-1].split('.')[0][-2:])
    return var
train_number_list = list(map(splitstr_int,train_df.values))
test_number_list = list(map(splitstr_int,test_df.values))
#因为一个训练样本包含41个图片时长为4h，考虑到一个seq就需要4h所以无法在样本内进行滑窗切割数据集，只需要返回初始下标即可，即x[1]==0
train_number_list = list(filter(lambda x:x[1]==0,enumerate(train_number_list)))
test_number_list = list(filter(lambda x:x[1]==0,enumerate(test_number_list)))
print(len(train_number_list))
print(len(test_number_list))






20000
3000


In [36]:
#下面给出一种筛选研究的方法demo即我只筛选20dbz回波面积占1/3以上的样本作为classfiy1（中等回波强度过程），即数据分析分型后在这个方法里修改
#判定条件，作为classfiy的数据集训练对应模型在inference的时候判断从而选择何种classfiy模型
import PIL.Image as Image
import numpy as np
def filter_pic(tid):
    tid = tid[0]
    frame = np.array(Image.open(os.path.join(self.rootdir, self._df['fname'][tid])))
    #缺测值为255我是直接将其滤成0不知道有没有其他好一点的办法，或者random一个0-80dbz的数值
    frame[frame>=80.5] = 0
    #print(frame)
    count  = (frame > 20)
    #print(self.threshold)
    counts = np.logical_and(count,True).sum()
    print(tid,'is processing')
    if counts >=self.area:
        x = tid
    else:
        x= None
    return x

#demo不进行任何操作直接返回
def demo_pic(tid):
    return tid[0]
with Pool(8) as p:
    train_list=list(p.map(demo_pic,train_number_list))
    test_list = list(p.map(demo_pic,train_number_list))
train_list = np.array(train_list)
test_list  = np.array(test_list)
#这样返回的索引就是筛选过后的
#index_list = list(filter(lambda x:x!=None,x))
train_savepath = '/media/workdir/hujh/hujh-new/huaweirader_baseline/data_eda/train_demo.npy'
test_savepath = '/media/workdir/hujh/hujh-new/huaweirader_baseline/data_eda/test_demo.npy'
save = np.save(train_savepath,train_list)
save = np.save(test_savepath,train_list)

In [39]:
import imageio
im = imageio.imread('/media/data/huaweiraderdata/train/train_all/RAD_184392561770451/RAD_184392561770451_000.png')

In [44]:
im.shape
im.max()

50